# Notebook para o PAN - Atribuição Autoral - 2018

In [1]:
!pip install spacy

  Using cached https://files.pythonhosted.org/packages/76/f4/7146c3812f96fcaaf2d06ff6862582302626a59011ccb6f2833bb38d80f7/regex-2018.01.10.tar.gz
  Complete output from command /anaconda3/bin/python -u -c "import setuptools, tokenize;__file__='/private/var/folders/w4/5hrb09795hx2z0_yw6r0sg780000gn/T/pip-install-b0ncj_5c/regex/setup.py';f=getattr(tokenize, 'open', open)(__file__);code=f.read().replace('\r\n', '\n');f.close();exec(compile(code, __file__, 'exec'))" bdist_wheel -d /private/var/folders/w4/5hrb09795hx2z0_yw6r0sg780000gn/T/pip-wheel-u9k3axmz --python-tag cp36:
  /anaconda3/lib/python3.6/site-packages/setuptools/dist.py:475: UserWarning: Normalizing '2018.01.10' to '2018.1.10'
    normalized_version,
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build/lib.macosx-10.7-x86_64-3.6
  copying regex_3/regex.py -> build/lib.macosx-10.7-x86_64-3.6
  copying regex_3/_regex_core.py -> build/lib.macosx-10.7-x86_64-3.6
  copying regex_3/test_regex.py

In [2]:
!python -m spacy download pt
!python -m spacy download en
!python -m spacy download es
!python -m spacy download de
!python -m spacy download fr
!python -m spacy download it


    Linking successful
    /anaconda3/lib/python3.6/site-packages/pt_core_news_sm -->
    /anaconda3/lib/python3.6/site-packages/spacy/data/pt

    You can now load the model via spacy.load('pt')


    Linking successful
    /anaconda3/lib/python3.6/site-packages/en_core_web_sm -->
    /anaconda3/lib/python3.6/site-packages/spacy/data/en

    You can now load the model via spacy.load('en')


    Linking successful
    /anaconda3/lib/python3.6/site-packages/es_core_news_sm -->
    /anaconda3/lib/python3.6/site-packages/spacy/data/es

    You can now load the model via spacy.load('es')


    Linking successful
    /anaconda3/lib/python3.6/site-packages/de_core_news_sm -->
    /anaconda3/lib/python3.6/site-packages/spacy/data/de

    You can now load the model via spacy.load('de')


    Linking successful
    /anaconda3/lib/python3.6/site-packages/fr_core_news_sm -->
    /anaconda3/lib/python3.6/site-packages/spacy/data/fr

    You can now load the model via spacy.load('fr')


    Linkin

In [3]:
%matplotlib inline
#python basic libs
import os;
from os.path import join as pathjoin;

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
from sklearn.exceptions import UndefinedMetricWarning
warnings.simplefilter(action='ignore', category=UndefinedMetricWarning)

import re;
import glob;
import json;
import codecs;
from collections import defaultdict;
import pprint;


from pprint import pprint
from time import time
import logging


#data analysis libs
import numpy as np;
import pandas as pd;
from pandas.plotting import scatter_matrix;
import matplotlib.pyplot as plt;
import random;

#machine learning libs
#feature extraction
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer, TfidfTransformer

#preprocessing and transformation
from sklearn.preprocessing import normalize, MaxAbsScaler, MinMaxScaler, Normalizer, RobustScaler;
from sklearn.preprocessing import LabelBinarizer;
from sklearn.decomposition import PCA;
from sklearn.metrics.pairwise import cosine_similarity;


from sklearn.base import BaseEstimator, ClassifierMixin

#classifiers
from sklearn.svm import LinearSVC, SVC
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier

from sklearn.feature_selection import RFE,SelectFpr,SelectPercentile, chi2;

#
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, VotingClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

#model valuation
from sklearn.model_selection import train_test_split;
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score, accuracy_score;

In [4]:
import seaborn as sns;
sns.set(color_codes=True);

In [5]:
import platform;
import sklearn;
import scipy;
import spacy;

print("|%-15s|%-40s|"%("PACK","VERSION"))
print("|%-15s|%-40s|"%('-'*15,'-'*40))
print('\n'.join(
    "|%-15s|%-40s|" % (pack, version)
    for pack, version in
    zip(['SO','NumPy','SciPy','Scikit-Learn','seaborn','spacy'],
        [platform.platform(), np.__version__, scipy.__version__, sklearn.__version__, sns.__version__, spacy.__version__])

))

|PACK           |VERSION                                 |
|---------------|----------------------------------------|
|SO             |Darwin-18.2.0-x86_64-i386-64bit         |
|NumPy          |1.15.4                                  |
|SciPy          |1.1.0                                   |
|Scikit-Learn   |0.20.1                                  |
|seaborn        |0.9.0                                   |
|spacy          |2.0.16                                  |


In [6]:
np.set_printoptions(precision=4)
pd.options.display.float_format = '{:,.4f}'.format

### paths configuration

In [9]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


In [10]:
!ls '/gdrive/My Drive/data_repository'

banking.csv.gz	out  pan18aa  Sentiment.csv.zip


In [0]:
baseDir = '/gdrive/My Drive/data_repository';
modelName = 'char1';

inputDir= pathjoin(baseDir,'pan18aa');
outputDir= pathjoin(baseDir,'out',modelName);

os.makedirs(outputDir, exist_ok=True)


In [7]:
baseDir = '/Users/joseeleandrocustodio/Dropbox/mestrado/02 - Pesquisa/code';

inputDir= pathjoin(baseDir,'pan18aa');
outputDir= pathjoin(baseDir,'out',"oficial");
if not os.path.exists(outputDir):
    os.mkdir(outputDir);

## Data Ingestion

In [0]:
def transferCode(filename):
  with open(os.path.join('/gdrive/My Drive','Colab Notebooks',filename)) as fin:
    with open(filename,'w') as fout:
      fout.write(fin.read());

In [0]:
transferCode('pan.py')
transferCode('sklearnExtensions.py')

In [16]:
!ls 

pan.py	sample_data  sklearnExtensions.py


In [8]:
import pan
import sklearnExtensions
from sklearnExtensions import DenseTransformer
from sklearnExtensions import POSTagTransformer

In [9]:
problems = pan.readCollectionsOfProblems(inputDir);

In [10]:
pd.DataFrame(problems)

,candidates,candidates_folder_count,encoding,language,problem,unknown
0,"[(graceful ones.\n\n""One more,"" Marvelous said...",20,UTF-8,en,problem00001,"[(after all, his best friends. And what in the..."
1,"[(a mission.""\n\nJensen just raises an eyebrow...",5,UTF-8,en,problem00002,"[(“Potter was attractive,” Draco thought, sigh..."
2,[(qui l'avait tué mais tout était de la faute ...,20,UTF-8,fr,problem00003,[(son réveil. Sa main pulse et Draco frotte l'...
3,[(. Le canapé est vide et lorsqu'il passe deva...,5,UTF-8,fr,problem00004,"[(abasourdie.\n\nTout d'abord, elle crut que s..."
4,"[(Eppure lui la mappa l’aveva stampata, dannaz...",20,UTF-8,it,problem00005,[(– Oh. Cazzo.\nSirius era così sconvolto che ...
5,[(Yato ha trovato una lettera sul suo comodino...,5,UTF-8,it,problem00006,"[(così la tua vista, Moony?\n– Cercavo di esse..."
6,[(zmienił zdanie. Niech się stworzonko pobawi....,20,UTF-8,pl,problem00007,"[(dawniej pełna radości i ciepła, a teraz wiec..."
7,"[(Słowem, które Sherlock najczęściej słyszał w...",5,UTF-8,pl,problem00008,"[(, uderzającego o żebra niczym dzwon- niemal ..."
8,[(pero no lo ama como ama a Guignol –explicó e...,20,UTF-8,sp,problem00009,[(–La nariz puntiaguda del elfo casi rozaba el...
9,"[(incapaz de señalar un momento exacto, un pun...",5,UTF-8,sp,problem00010,[(tan parecidas hizo que su trasero latiese de...


In [11]:
def cachingPOSTAG(problem, taggingVersion='TAG'):
    import json;
    cachePath = os.path.join(outputDir,'POSTAG_cache');
    if not os.path.exists(cachePath):
        os.makedirs(cachePath);
    
    _id = problem['problem']+problem['language'];
    filename = os.path.join(cachePath,taggingVersion+'_'+_id+'.json')
    if not os.path.exists(filename):
        lang = problem['language'];
        if lang == 'sp':
            lang = 'es';
        elif lang =='pl':
            print(lang, ' not supported');
            return ;

        train_docs, train_labels, _   = zip(*problem['candidates'])
        problem['training_docs_size'] = len(train_docs);
        test_docs, _, test_filename   = zip(*problem['unknown'])

        t0 = time()
        tagger = POSTagTransformer(language=lang);
        train_docs = tagger.fit_transform(train_docs);
        test_docs = tagger.fit_transform(test_docs);
        
        print("Annotation time %0.3fs" % (time() - t0))
        
        with open(filename,'w') as f:
            json.dump({
                'train':train_docs,
                'train_labels':train_labels,
                'test':test_docs,
                'test_filename':test_filename
            },f);
    else:
        with open(filename,'r') as f:
            data = json.load(f);

            train_docs = data['train'];
            train_labels = data['train_labels'];
            test_docs  = data['test'];
            test_filename = data['test_filename'];
    return train_docs, train_labels, test_docs, test_filename;

for problem in problems:
    cachingPOSTAG(problem)

Annotation time 39.187s
Annotation time 10.132s
Annotation time 42.915s
Annotation time 18.518s
Annotation time 36.385s
Annotation time 13.330s
pl  not supported
pl  not supported
Annotation time 43.873s
Annotation time 14.763s


In [14]:
class FilterTagTransformer(BaseEstimator):
    def __init__(self,token='POS', parts=None,verbose=False):
        self.token = token;
        self.parts = parts;
        self.verbose = verbose;
            

    def transform(self, X, y=None):
        """ Return An array of tokens 
        Parameters
        ----------
        X : {array-like}, shape = [n_samples, n_tokens]
            Array documents, where each document consists of a list of node
            and each node consist of a token and its correspondent tag
            
            [
            [('a','TAG1'),('b','TAG2')],
            [('a','TAG1')]
            ]
        y : array-like, shape = [n_samples] (default: None)
        Returns
        ---------
        X_dense : dense version of the input X array.
        """
        t0 = time()
        if self.verbose:
            print('filtered in', end=' ')
        if self.token == 'TAG':
            X = [' '.join([d[1].split('__')[0] for d in doc]) for doc in X]
        elif self.token == 'POS':
            if self.parts is None:
                X = [' '.join([d[2] for d in doc]) for doc in X];
            else:
                X = [' '.join([d[0] for d in doc if d[2] in self.parts]) for doc in X]
        elif self.token == 'DEP':
            X = [' '.join([d[3] for d in doc]) for doc in X]
        elif self.token == 'word_POS':
            if self.parts is None:
                X = [' '.join([d[0]+'/'+d[2] for d in doc]) for doc in X]
        elif self.token == 'filter':
            X = [' '.join([d[0] for d in doc if d[2] in self.parts]) for doc in X]
        else:
            X = [' '.join([d[0] for d in doc]) for doc in X]
        
        if self.verbose:
            print(" %0.3fs" % (time() - t0))
        return np.array(X);       

    def fit(self, X, y=None):
        self.is_fitted = True
        return self

    def fit_transform(self, X, y=None):
        return self.transform(X=X, y=y)

In [15]:
def  listTag(problem):
    print ("\nProblem: %s,  language: %s, " %(problem['problem'],problem['language']), end=' ');
    
    lang = problem['language'];
    if lang == 'sp':
        lang = 'es';
    elif lang =='pl':
        print(lang, ' not supported');
        return None;
    
    
    train_docs, train_labels, test_docs, test_filename = cachingPOSTAG(problem)
    
    detectedPOS = sorted(set([d[2] for doc in train_docs for d in doc if d[2] not in ['','SPACE','SYM']]))
    
    print("Detected POS: \n"+ str(detectedPOS));
for problem in problems:
    listTag(problem)


Problem: problem00001,  language: en,  Detected POS: 
['ADJ', 'ADP', 'ADV', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'VERB', 'X']

Problem: problem00002,  language: en,  Detected POS: 
['ADJ', 'ADP', 'ADV', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'VERB', 'X']

Problem: problem00003,  language: fr,  Detected POS: 
['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'VERB', 'X']

Problem: problem00004,  language: fr,  Detected POS: 
['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'VERB', 'X']

Problem: problem00005,  language: it,  Detected POS: 
['ADJ', 'ADP', 'ADV', 'AUX', 'CONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'VERB', 'X']

Problem: problem00006,  language: it,  Detected POS: 
['ADJ', 'ADP', 'ADV', 'AUX', 'CONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PRON', 'PROPN', 'PUNCT'

In [16]:
def countFeatures(taggedDocs,min_df,ngram_range=(2,5)):
    detectedPOS = sorted(set([d[2] for doc in taggedDocs for d in doc]));
    counts = [];
    
    for p in detectedPOS:
        pipeline = Pipeline([
                  ('filter',FilterTagTransformer(token='filter',parts=[p])),
                  ('vect', CountVectorizer(
                  analyzer='char_wb',
                  ngram_range=ngram_range,
                  lowercase=False
                  ))
                 ]);
        try:
            validFeatures = np.sum(np.mean(pipeline.fit_transform(taggedDocs)>0, axis=0) > min_df);
        except ValueError:
            validFeatures =0;
        counts.append(validFeatures);
            
    return zip(detectedPOS,counts);


In [17]:
def runEachPOS(problem):
    print ("\nProblem: %s,  language: %s, " %(problem['problem'],problem['language']), end=' ');
    
    lang = problem['language'];
    if lang == 'sp':
        lang = 'es';
    elif lang =='pl':
        print(lang, ' not supported');
        return None,None;
    
    
    train_docs, train_labels, test_docs, test_filename = cachingPOSTAG(problem)
    
    detectedPOS = countFeatures(train_docs,0.01,(1,5));
    
    POS = [p[0] for p in detectedPOS if p[1] > 100];
    print("Detected POS: "+ str(POS));
    
    t0 = time()
    
    pipeline = Pipeline([
        ('filter',FilterTagTransformer(token='filter')),
        ('vect', CountVectorizer(
                                   analyzer='char_wb',
                                   ngram_range=(2,5),
                                   min_df=0.01,
                                   lowercase=False
                                  )),
        ('tfidf',  TfidfTransformer(norm='l2',sublinear_tf=True)),
        ('scaler', MaxAbsScaler()),
        ('dense',  DenseTransformer()),
        ('transf', PCA(0.999)),
        ('clf', LogisticRegression(random_state=0,multi_class='multinomial', solver='newton-cg')),
    ])
    
    
    # uncommenting more parameters will give better exploring power but will
    # increase processing time in a combinatorial way
    parameters = {
        'filter__parts':POS
    }
    
    grid_search = GridSearchCV(pipeline,
                               parameters,
                               cv=4,
                               iid=False,
                               n_jobs=-1,
                               verbose=True,
                               scoring='f1_macro')
    
    t0 = time()
    grid_search.fit(train_docs, train_labels)
    
    print("Gridsearh %0.3fs" % (time() - t0), end=' ')

    print("Best score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
        
    train_pred=grid_search.predict(train_docs);
    test_pred=grid_search.predict(test_docs);
    
    
    # Writing output file
    out_data=[]
    for i,v in enumerate(test_pred):
        out_data.append({'unknown-text': test_filename[i],'predicted-author': v})
    answerFile = pathjoin(outputDir,'answers-'+problem['problem']+'.json');
    with open(answerFile, 'w') as f:
        json.dump(out_data, f, indent=4)
    
    
    #calculating the performance using PAN evaluation code
    f1,precision,recall,accuracy=pan.evaluate(
                pathjoin(inputDir, problem['problem'], 'ground-truth.json'),
                answerFile)
    
    return  {
                'problem-name'  :   problem['problem'],
                "language"      :  problem['language'],
                'AuthorCount'   :  len(set(train_labels)),
                'macro-f1'       : round(f1,3),
                'macro-precision': round(precision,3),
                'macro-recall'   : round(recall,3),
                'micro-accuracy' : round(accuracy,3),
                
        },grid_search.cv_results_;

In [18]:
cv_result = [];
result = [];
for problem in problems:
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore");
        r,c = runEachPOS(problem);
        cv_result.append(c);
        result.append(r);



Problem: problem00001,  language: en,  Detected POS: ['ADJ', 'ADP', 'ADV', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'VERB']
Fitting 4 folds for each of 13 candidates, totalling 52 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done  52 out of  52 | elapsed:  2.7min finished


Gridsearh 163.726s Best score: 0.502
Best parameters set:
	filter__parts: 'PUNCT'

Problem: problem00002,  language: en,  Detected POS: ['ADJ', 'ADP', 'ADV', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'VERB']
Fitting 4 folds for each of 13 candidates, totalling 52 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   29.0s
[Parallel(n_jobs=-1)]: Done  52 out of  52 | elapsed:   35.5s finished


Gridsearh 35.546s Best score: 0.800
Best parameters set:
	filter__parts: 'PUNCT'

Problem: problem00003,  language: fr,  Detected POS: ['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'NOUN', 'NUM', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'VERB', 'X']
Fitting 4 folds for each of 14 candidates, totalling 56 fits


...

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done  56 out of  56 | elapsed:  2.7min finished


Gridsearh 163.600s Best score: 0.569
Best parameters set:
	filter__parts: 'PROPN'

Problem: problem00010,  language: sp,  Detected POS: ['ADJ', 'ADP', 'ADV', 'AUX', 'CONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'VERB']
Fitting 4 folds for each of 14 candidates, totalling 56 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   31.7s


Gridsearh 43.140s Best score: 0.922
Best parameters set:
	filter__parts: 'PROPN'


[Parallel(n_jobs=-1)]: Done  56 out of  56 | elapsed:   43.1s finished


In [19]:
languages={
    'en':'inglesa',
    'sp':'espanhola',
    'it':'italiana',
    'pl':'polonesa',
    'fr':'francesa'
}

In [20]:
cv_result_parts = cv_result;
dfParts = pd.DataFrame();
for i, c in enumerate(cv_result_parts):
    if c is None:
        continue;
    temp = pd.DataFrame(c);
    temp['problem'] = int(re.sub('\D','',result[i]['problem-name']));
    temp['f1best']  = result[i]['macro-f1'];
    temp['language'] = languages[result[i]['language']];
    dfParts = dfParts.append(temp);

for p in [
    'mean_test_score','std_test_score','mean_train_score',   
    'split0_test_score',
    'split1_test_score',
    'split2_test_score']:
    dfParts[p]=dfParts[p].astype(np.float32);

    
dfParts =dfParts[[
    'problem',
    'language',
    'f1best',
    'rank_test_score',
    'param_filter__parts',
    'mean_test_score',   
    'std_test_score',
    'mean_score_time',
    'mean_fit_time',
    'std_fit_time',
    'std_score_time',
    'std_train_score',
]];

dfParts.rename(columns={
    'param_filter__parts':'POSFILTER',
},inplace=True);

#print('\',\n\''.join(dfCV.columns))


In [21]:
dfParts.sort_values(['problem', 'rank_test_score'], inplace=True)
dfParts

,problem,language,f1best,rank_test_score,POSFILTER,mean_test_score,std_test_score,mean_score_time,mean_fit_time,std_fit_time,std_score_time,std_train_score
11,1,inglesa,0.3010,1,PUNCT,0.5021,0.0811,0.0170,0.0646,0.0038,0.0038,0.0339
10,1,inglesa,0.3010,2,PROPN,0.4600,0.0383,0.0613,0.4281,0.2202,0.0329,0.0000
12,1,inglesa,0.3010,3,VERB,0.3982,0.0310,0.1033,0.4714,0.0137,0.0311,0.0000
6,1,inglesa,0.3010,4,NOUN,0.3813,0.1015,0.0990,0.6101,0.0642,0.0322,0.0000
2,1,inglesa,0.3010,5,ADV,0.2558,0.0285,0.0600,0.2491,0.0520,0.0279,0.0000
0,1,inglesa,0.3010,6,ADJ,0.2404,0.0667,0.0473,0.2823,0.0186,0.0127,0.0000
1,1,inglesa,0.3010,7,ADP,0.2185,0.0597,0.0294,0.1193,0.0291,0.0091,0.0000
5,1,inglesa,0.3010,8,INTJ,0.1356,0.0298,0.0113,0.0531,0.0081,0.0039,0.0356
9,1,inglesa,0.3010,9,PRON,0.0962,0.0536,0.0443,0.1659,0.0351,0.0099,0.0546
4,1,inglesa,0.3010,10,DET,0.0861,0.0328,0.0173,0.0715,0.0047,0.0043,0.0340


In [23]:
import seaborn as sns

cm = sns.light_palette("green", as_cmap=True);

with pd.option_context('display.precision', 3):
    html = dfParts.pivot_table(
                index=['POSFILTER'],
                columns=['problem','language'],
                values=['mean_test_score']
            ).style.background_gradient(cmap=cm).highlight_null('#fcc')
html

In [24]:
dfParts.to_csv('PANAA2018_POSCHAR_ONEPOS.csv', index=False)

### analisando os demais parametros

In [25]:
dfParts.sort_values(['problem', 'rank_test_score'], inplace=True)
bestPOS = {};
for i,pos in zip(dfParts.problem.values,dfParts.POSFILTER.values):
    if i not in bestPOS:
        bestPOS[i] =[];
    bestPOS[i].append(pos);
    
for i in bestPOS.keys():
    print(i,'==>', bestPOS[i])

1 ==> ['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV', 'ADJ', 'ADP', 'INTJ', 'PRON', 'DET', 'CCONJ', 'NUM', 'PART']
2 ==> ['PUNCT', 'VERB', 'ADV', 'PROPN', 'NOUN', 'NUM', 'ADJ', 'PRON', 'INTJ', 'ADP', 'PART', 'DET', 'CCONJ']
3 ==> ['NOUN', 'VERB', 'PROPN', 'ADV', 'ADJ', 'PRON', 'PUNCT', 'ADP', 'DET', 'AUX', 'CCONJ', 'X', 'SCONJ', 'NUM']
4 ==> ['NOUN', 'ADV', 'VERB', 'PROPN', 'ADJ', 'X', 'SCONJ', 'DET', 'PRON', 'AUX', 'CCONJ', 'PUNCT', 'NUM', 'ADP']
5 ==> ['VERB', 'NOUN', 'PUNCT', 'PROPN', 'ADV', 'ADJ', 'PRON', 'DET', 'SCONJ', 'AUX', 'CONJ', 'X', 'ADP', 'INTJ', 'NUM']
6 ==> ['VERB', 'PROPN', 'PUNCT', 'ADV', 'NOUN', 'PRON', 'SCONJ', 'AUX', 'ADJ', 'NUM', 'CONJ', 'DET', 'INTJ', 'ADP', 'X']
9 ==> ['PROPN', 'VERB', 'NOUN', 'PUNCT', 'ADJ', 'ADV', 'PRON', 'AUX', 'SCONJ', 'ADP', 'CONJ', 'DET', 'INTJ', 'NUM']
10 ==> ['PROPN', 'VERB', 'NOUN', 'ADJ', 'ADV', 'SCONJ', 'ADP', 'PUNCT', 'PRON', 'DET', 'CONJ', 'AUX', 'NUM', 'INTJ']


In [26]:
#creating subset with best features first

for i in range(0,len(bestPOS[1])):
    print(bestPOS[1][0:(i+1)])

['PUNCT']
['PUNCT', 'PROPN']
['PUNCT', 'PROPN', 'VERB']
['PUNCT', 'PROPN', 'VERB', 'NOUN']
['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV']
['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV', 'ADJ']
['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV', 'ADJ', 'ADP']
['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV', 'ADJ', 'ADP', 'INTJ']
['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV', 'ADJ', 'ADP', 'INTJ', 'PRON']
['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV', 'ADJ', 'ADP', 'INTJ', 'PRON', 'DET']
['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV', 'ADJ', 'ADP', 'INTJ', 'PRON', 'DET', 'CCONJ']
['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV', 'ADJ', 'ADP', 'INTJ', 'PRON', 'DET', 'CCONJ', 'NUM']
['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV', 'ADJ', 'ADP', 'INTJ', 'PRON', 'DET', 'CCONJ', 'NUM', 'PART']


In [27]:
def runML(problem, sortedPOS):
    import gc;
    gc.collect();
    print ("\nProblem: %s,  language: %s, " %(problem['problem'],problem['language']));
    
    lang = problem['language'];
    if lang == 'sp':
        lang = 'es';
    elif lang =='pl':
        print(lang, ' not supported');
        return None,None,None,None;
    
    
    train_docs, train_labels, test_docs, test_filename = cachingPOSTAG(problem)
    
    POS = [
        sortedPOS[0:(i+1)]
        for i in range(0, len(sortedPOS))
    ]
    for p in POS:
        print(p);
    
    t0 = time()
    
    pipeline = Pipeline([
        ('filter',FilterTagTransformer(token='filter')),
        ('vect', TfidfVectorizer(
                                   analyzer='char_wb',
                                   min_df=0.01,
                                   lowercase=False,
                                    norm='l2',sublinear_tf=True)),
        ('scaler', MaxAbsScaler()),
        ('dense',  DenseTransformer()),
        ('transf', PCA(0.999)),
        ('clf', LogisticRegression(random_state=0,multi_class='multinomial', solver='newton-cg')),
    ])
    
    
    # uncommenting more parameters will give better exploring power but will
    # increase processing time in a combinatorial way
    parameters = {
        'filter__parts':POS,
        'vect__ngram_range':((2,5),(2,4),(3,5))
    }
    
    grid_search = GridSearchCV(pipeline,
                               parameters,
                               cv=3,
                               iid=False,
                               n_jobs=1,
                               verbose=True,
                               scoring='f1_macro')
    
    t0 = time()
    grid_search.fit(train_docs, train_labels)
    
    print("Gridsearh %0.3fs" % (time() - t0), end=' ')

    print("Best score: %0.3f" % grid_search.best_score_)
    print("Best parameters set:")
    best_parameters = grid_search.best_estimator_.get_params()
    for param_name in sorted(parameters.keys()):
        print("\t%s: %r" % (param_name, best_parameters[param_name]))
        
    train_pred=grid_search.predict(train_docs);
    test_pred=grid_search.predict(test_docs);
    
    
    # Writing output file
    out_data=[]
    for i,v in enumerate(test_pred):
        out_data.append({'unknown-text': test_filename[i],'predicted-author': v})
    answerFile = pathjoin(outputDir,'answers-'+problem['problem']+'.json');
    with open(answerFile, 'w') as f:
        json.dump(out_data, f, indent=4)
    
    
    #calculating the performance using PAN evaluation code
    f1,precision,recall,accuracy=pan.evaluate(
                pathjoin(inputDir, problem['problem'], 'ground-truth.json'),
                answerFile)
    
    return {
                'problem-name'  :       problem['problem'],
                "language"      :       problem['language'],
                'macro-f1'       : round(f1,3),
                'macro-precision': round(precision,3),
                'macro-recall'   : round(recall,3),
                'micro-accuracy' : round(accuracy,3),
                
        }, grid_search.cv_results_,best_parameters, grid_search.best_estimator_;

In [29]:
result = [];
cv_result = [];
best_parameters = [];
estimators = [];
with warnings.catch_warnings():
    warnings.filterwarnings("ignore");
    for problem in problems:
        i = int(re.sub('\D+','',problem['problem']));
        if i not in bestPOS:
            continue;
        r, c, b, e = runML(problem,bestPOS[i]);
        result.append(r);
        cv_result.append(c);
        estimators.append(e);
        b['problem'] = problem['problem'];
        best_parameters.append(b);


Problem: problem00001,  language: en, 
['PUNCT']
['PUNCT', 'PROPN']
['PUNCT', 'PROPN', 'VERB']
['PUNCT', 'PROPN', 'VERB', 'NOUN']
['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV']
['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV', 'ADJ']
['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV', 'ADJ', 'ADP']
['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV', 'ADJ', 'ADP', 'INTJ']
['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV', 'ADJ', 'ADP', 'INTJ', 'PRON']
['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV', 'ADJ', 'ADP', 'INTJ', 'PRON', 'DET']
['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV', 'ADJ', 'ADP', 'INTJ', 'PRON', 'DET', 'CCONJ']
['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV', 'ADJ', 'ADP', 'INTJ', 'PRON', 'DET', 'CCONJ', 'NUM']
['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV', 'ADJ', 'ADP', 'INTJ', 'PRON', 'DET', 'CCONJ', 'NUM', 'PART']
Fitting 3 folds for each of 39 candidates, totalling 117 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 117 out of 117 | elapsed:  2.7min finished


Gridsearh 164.907s Best score: 0.796
Best parameters set:
	filter__parts: ['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV', 'ADJ', 'ADP', 'INTJ', 'PRON', 'DET', 'CCONJ', 'NUM', 'PART']
	vect__ngram_range: (2, 4)

Problem: problem00002,  language: en, 
['PUNCT']
['PUNCT', 'VERB']
['PUNCT', 'VERB', 'ADV']
['PUNCT', 'VERB', 'ADV', 'PROPN']
['PUNCT', 'VERB', 'ADV', 'PROPN', 'NOUN']
['PUNCT', 'VERB', 'ADV', 'PROPN', 'NOUN', 'NUM']
['PUNCT', 'VERB', 'ADV', 'PROPN', 'NOUN', 'NUM', 'ADJ']
['PUNCT', 'VERB', 'ADV', 'PROPN', 'NOUN', 'NUM', 'ADJ', 'PRON']
['PUNCT', 'VERB', 'ADV', 'PROPN', 'NOUN', 'NUM', 'ADJ', 'PRON', 'INTJ']
['PUNCT', 'VERB', 'ADV', 'PROPN', 'NOUN', 'NUM', 'ADJ', 'PRON', 'INTJ', 'ADP']
['PUNCT', 'VERB', 'ADV', 'PROPN', 'NOUN', 'NUM', 'ADJ', 'PRON', 'INTJ', 'ADP', 'PART']
['PUNCT', 'VERB', 'ADV', 'PROPN', 'NOUN', 'NUM', 'ADJ', 'PRON', 'INTJ', 'ADP', 'PART', 'DET']
['PUNCT', 'VERB', 'ADV', 'PROPN', 'NOUN', 'NUM', 'ADJ', 'PRON', 'INTJ', 'ADP', 'PART', 'DET', 'CCONJ']
Fitting 3 folds for eac

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 117 out of 117 | elapsed:   35.7s finished


Gridsearh 35.823s Best score: 0.977
Best parameters set:
	filter__parts: ['PUNCT', 'VERB', 'ADV']
	vect__ngram_range: (2, 5)

Problem: problem00003,  language: fr, 
['NOUN']
['NOUN', 'VERB']
['NOUN', 'VERB', 'PROPN']
['NOUN', 'VERB', 'PROPN', 'ADV']
['NOUN', 'VERB', 'PROPN', 'ADV', 'ADJ']
['NOUN', 'VERB', 'PROPN', 'ADV', 'ADJ', 'PRON']
['NOUN', 'VERB', 'PROPN', 'ADV', 'ADJ', 'PRON', 'PUNCT']
['NOUN', 'VERB', 'PROPN', 'ADV', 'ADJ', 'PRON', 'PUNCT', 'ADP']
['NOUN', 'VERB', 'PROPN', 'ADV', 'ADJ', 'PRON', 'PUNCT', 'ADP', 'DET']
['NOUN', 'VERB', 'PROPN', 'ADV', 'ADJ', 'PRON', 'PUNCT', 'ADP', 'DET', 'AUX']
['NOUN', 'VERB', 'PROPN', 'ADV', 'ADJ', 'PRON', 'PUNCT', 'ADP', 'DET', 'AUX', 'CCONJ']
['NOUN', 'VERB', 'PROPN', 'ADV', 'ADJ', 'PRON', 'PUNCT', 'ADP', 'DET', 'AUX', 'CCONJ', 'X']
['NOUN', 'VERB', 'PROPN', 'ADV', 'ADJ', 'PRON', 'PUNCT', 'ADP', 'DET', 'AUX', 'CCONJ', 'X', 'SCONJ']
['NOUN', 'VERB', 'PROPN', 'ADV', 'ADJ', 'PRON', 'PUNCT', 'ADP', 'DET', 'AUX', 'CCONJ', 'X', 'SCONJ', 'NUM']
Fitt

...

Gridsearh 51.334s Best score: 0.919
Best parameters set:
	filter__parts: ['VERB', 'PROPN', 'PUNCT', 'ADV', 'NOUN', 'PRON']
	vect__ngram_range: (2, 5)

Problem: problem00009,  language: sp, 
['PROPN']
['PROPN', 'VERB']
['PROPN', 'VERB', 'NOUN']
['PROPN', 'VERB', 'NOUN', 'PUNCT']
['PROPN', 'VERB', 'NOUN', 'PUNCT', 'ADJ']
['PROPN', 'VERB', 'NOUN', 'PUNCT', 'ADJ', 'ADV']
['PROPN', 'VERB', 'NOUN', 'PUNCT', 'ADJ', 'ADV', 'PRON']
['PROPN', 'VERB', 'NOUN', 'PUNCT', 'ADJ', 'ADV', 'PRON', 'AUX']
['PROPN', 'VERB', 'NOUN', 'PUNCT', 'ADJ', 'ADV', 'PRON', 'AUX', 'SCONJ']
['PROPN', 'VERB', 'NOUN', 'PUNCT', 'ADJ', 'ADV', 'PRON', 'AUX', 'SCONJ', 'ADP']
['PROPN', 'VERB', 'NOUN', 'PUNCT', 'ADJ', 'ADV', 'PRON', 'AUX', 'SCONJ', 'ADP', 'CONJ']
['PROPN', 'VERB', 'NOUN', 'PUNCT', 'ADJ', 'ADV', 'PRON', 'AUX', 'SCONJ', 'ADP', 'CONJ', 'DET']
['PROPN', 'VERB', 'NOUN', 'PUNCT', 'ADJ', 'ADV', 'PRON', 'AUX', 'SCONJ', 'ADP', 'CONJ', 'DET', 'INTJ']
['PROPN', 'VERB', 'NOUN', 'PUNCT', 'ADJ', 'ADV', 'PRON', 'AUX', 'SCONJ

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 126 out of 126 | elapsed:  3.6min finished


Gridsearh 220.135s Best score: 0.874
Best parameters set:
	filter__parts: ['PROPN', 'VERB', 'NOUN', 'PUNCT', 'ADJ', 'ADV', 'PRON', 'AUX', 'SCONJ', 'ADP', 'CONJ', 'DET']
	vect__ngram_range: (2, 4)

Problem: problem00010,  language: sp, 
['PROPN']
['PROPN', 'VERB']
['PROPN', 'VERB', 'NOUN']
['PROPN', 'VERB', 'NOUN', 'ADJ']
['PROPN', 'VERB', 'NOUN', 'ADJ', 'ADV']
['PROPN', 'VERB', 'NOUN', 'ADJ', 'ADV', 'SCONJ']
['PROPN', 'VERB', 'NOUN', 'ADJ', 'ADV', 'SCONJ', 'ADP']
['PROPN', 'VERB', 'NOUN', 'ADJ', 'ADV', 'SCONJ', 'ADP', 'PUNCT']
['PROPN', 'VERB', 'NOUN', 'ADJ', 'ADV', 'SCONJ', 'ADP', 'PUNCT', 'PRON']
['PROPN', 'VERB', 'NOUN', 'ADJ', 'ADV', 'SCONJ', 'ADP', 'PUNCT', 'PRON', 'DET']
['PROPN', 'VERB', 'NOUN', 'ADJ', 'ADV', 'SCONJ', 'ADP', 'PUNCT', 'PRON', 'DET', 'CONJ']
['PROPN', 'VERB', 'NOUN', 'ADJ', 'ADV', 'SCONJ', 'ADP', 'PUNCT', 'PRON', 'DET', 'CONJ', 'AUX']
['PROPN', 'VERB', 'NOUN', 'ADJ', 'ADV', 'SCONJ', 'ADP', 'PUNCT', 'PRON', 'DET', 'CONJ', 'AUX', 'NUM']
['PROPN', 'VERB', 'NOUN', 'AD

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 126 out of 126 | elapsed:   49.8s finished


Gridsearh 49.910s Best score: 0.933
Best parameters set:
	filter__parts: ['PROPN', 'VERB']
	vect__ngram_range: (2, 5)


In [32]:
df=pd.DataFrame(result)[['problem-name',
                     "language",
                     'macro-f1','macro-precision','macro-recall' ,'micro-accuracy']]

In [33]:
df

,problem-name,language,macro-f1,macro-precision,macro-recall,micro-accuracy
0,problem00001,en,0.5640,0.5550,0.6790,0.6860
1,problem00002,en,0.5330,0.5240,0.6830,0.6190
2,problem00003,fr,0.6360,0.6380,0.7020,0.6730
3,problem00004,fr,0.7480,0.8080,0.7730,0.7140
4,problem00005,it,0.5310,0.5070,0.6780,0.6500
5,problem00006,it,0.5180,0.5580,0.6270,0.7170
6,problem00009,sp,0.7690,0.7600,0.8360,0.7860
7,problem00010,sp,0.6150,0.6500,0.6470,0.6880


In [34]:
print(df[["macro-f1"]].reset_index().to_latex(index=False).replace("     "," "))

\begin{tabular}{rr}
\toprule
 index &  macro-f1 \\
\midrule
 0 &    0.5640 \\
 1 &    0.5330 \\
 2 &    0.6360 \\
 3 &    0.7480 \\
 4 &    0.5310 \\
 5 &    0.5180 \\
 6 &    0.7690 \\
 7 &    0.6150 \\
\bottomrule
\end{tabular}



In [35]:
languages={
    'en':'inglesa',
    'sp':'espanhola',
    'it':'italiana',
    'pl':'polonesa',
    'fr':'francesa'
}

In [87]:
cv_result2 = [];
dfCV = pd.DataFrame();
for i, c in enumerate(cv_result):
    temp = pd.DataFrame(c);
    temp['problem'] = result[i]['problem-name'];
    temp['language'] = languages[result[i]['language']]
    dfCV = dfCV.append(temp);

for p in [
    'mean_test_score','std_test_score','mean_train_score',   
    'split0_test_score','split0_train_score',
    'split1_test_score','split1_train_score',
    'split2_test_score','split2_train_score']:
    dfCV[p]=dfCV[p].astype(np.float32);

    
dfCV =dfCV[[
    'problem',
    'language',
    'rank_test_score',
    'param_filter__parts',
    'param_vect__ngram_range',
    'mean_test_score',   
    'std_test_score',
    'mean_train_score',   

    'split0_test_score','split0_train_score',
    'split1_test_score','split1_train_score',
    'split2_test_score','split2_train_score',

    'mean_score_time',
    'mean_fit_time',
    'std_fit_time',
    'std_score_time',
    'std_train_score',
]];

dfCV.rename(columns={
    'param_filter__parts':'parts',
    'param_vect__ngram_range':'ngram_range',
},inplace=True);

def nrange(x):
    return sum([i for i in range(x+1)])

dfCV['rangeLen'] = dfCV.ngram_range.apply(lambda x: nrange(x[1])-nrange(x[0]-1))

#print('\',\n\''.join(dfCV.columns))

In [88]:
dfCV.to_csv('PANAA2018_POSCHAR.csv', index=False)

In [89]:
dfCV = pd.read_csv('PANAA2018_POSCHAR.csv', na_values='')

In [123]:
pd.options.display.max_colwidth = -1
(dfCV[dfCV.rank_test_score == 1])[
    ['problem',
     'language',
    'rank_test_score',
    'mean_test_score',
    'std_test_score',
    'ngram_range',
     'rangeLen',
    'parts']
].sort_values(by=[
    'problem',
    'mean_test_score',
    'parts',
    'rangeLen',
    'ngram_range',
], ascending=[True, False,True,True, True]).head(10)

,problem,language,rank_test_score,mean_test_score,std_test_score,ngram_range,rangeLen,parts
37,problem00001,inglesa,1,0.7964,0.0511,"(2, 4)",9,"['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV', 'ADJ', 'ADP', 'INTJ', 'PRON', 'DET', 'CCONJ', 'NUM', 'PART']"
65,problem00002,inglesa,1,0.9771,0.0323,"(3, 5)",12,"['PUNCT', 'VERB', 'ADV', 'PROPN', 'NOUN', 'NUM', 'ADJ', 'PRON', 'INTJ']"
63,problem00002,inglesa,1,0.9771,0.0323,"(2, 5)",14,"['PUNCT', 'VERB', 'ADV', 'PROPN', 'NOUN', 'NUM', 'ADJ', 'PRON', 'INTJ']"
61,problem00002,inglesa,1,0.9771,0.0323,"(2, 4)",9,"['PUNCT', 'VERB', 'ADV', 'PROPN', 'NOUN', 'NUM', 'ADJ', 'PRON']"
62,problem00002,inglesa,1,0.9771,0.0323,"(3, 5)",12,"['PUNCT', 'VERB', 'ADV', 'PROPN', 'NOUN', 'NUM', 'ADJ', 'PRON']"
60,problem00002,inglesa,1,0.9771,0.0323,"(2, 5)",14,"['PUNCT', 'VERB', 'ADV', 'PROPN', 'NOUN', 'NUM', 'ADJ', 'PRON']"
58,problem00002,inglesa,1,0.9771,0.0323,"(2, 4)",9,"['PUNCT', 'VERB', 'ADV', 'PROPN', 'NOUN', 'NUM', 'ADJ']"
59,problem00002,inglesa,1,0.9771,0.0323,"(3, 5)",12,"['PUNCT', 'VERB', 'ADV', 'PROPN', 'NOUN', 'NUM', 'ADJ']"
57,problem00002,inglesa,1,0.9771,0.0323,"(2, 5)",14,"['PUNCT', 'VERB', 'ADV', 'PROPN', 'NOUN', 'NUM', 'ADJ']"
49,problem00002,inglesa,1,0.9771,0.0323,"(2, 4)",9,"['PUNCT', 'VERB', 'ADV', 'PROPN']"


In [125]:
pd.set_option('display.max_colwidth', -1)
t=(dfCV)[
    ['problem',
     'language',
    'rank_test_score',
    'mean_test_score',
    'std_test_score',
    'ngram_range',
     'rangeLen',
    'parts']
].sort_values(by=[
    'problem',
    'rank_test_score',
    'parts',
    'rangeLen',
    'ngram_range',
], ascending=[True, True,True,True, True])
t

,problem,language,rank_test_score,mean_test_score,std_test_score,ngram_range,rangeLen,parts
37,problem00001,inglesa,1,0.7964,0.0511,"(2, 4)",9,"['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV', 'ADJ', 'ADP', 'INTJ', 'PRON', 'DET', 'CCONJ', 'NUM', 'PART']"
34,problem00001,inglesa,2,0.7880,0.0583,"(2, 4)",9,"['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV', 'ADJ', 'ADP', 'INTJ', 'PRON', 'DET', 'CCONJ', 'NUM']"
28,problem00001,inglesa,3,0.7863,0.0611,"(2, 4)",9,"['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV', 'ADJ', 'ADP', 'INTJ', 'PRON', 'DET']"
36,problem00001,inglesa,4,0.7800,0.0765,"(2, 5)",14,"['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV', 'ADJ', 'ADP', 'INTJ', 'PRON', 'DET', 'CCONJ', 'NUM', 'PART']"
25,problem00001,inglesa,5,0.7786,0.0555,"(2, 4)",9,"['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV', 'ADJ', 'ADP', 'INTJ', 'PRON']"
31,problem00001,inglesa,6,0.7729,0.0621,"(2, 4)",9,"['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV', 'ADJ', 'ADP', 'INTJ', 'PRON', 'DET', 'CCONJ']"
33,problem00001,inglesa,7,0.7712,0.0653,"(2, 5)",14,"['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV', 'ADJ', 'ADP', 'INTJ', 'PRON', 'DET', 'CCONJ', 'NUM']"
27,problem00001,inglesa,7,0.7712,0.0653,"(2, 5)",14,"['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV', 'ADJ', 'ADP', 'INTJ', 'PRON', 'DET']"
24,problem00001,inglesa,9,0.7708,0.0658,"(2, 5)",14,"['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV', 'ADJ', 'ADP', 'INTJ', 'PRON']"
16,problem00001,inglesa,10,0.7649,0.0318,"(2, 4)",9,"['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV', 'ADJ']"


In [121]:
pd.options.display.precision = 3
pd.options.display.max_colwidth = -1
t =(dfCV)[
    ['problem',
     'language',
    'rank_test_score',
    'mean_test_score',
    'ngram_range',
     'rangeLen',
    'parts']
].sort_values(by=[
    'problem',
    'rank_test_score',
    'parts',
    'rangeLen',
    'ngram_range',
], ascending=[True, True,True,True, True])\
.groupby(['problem']).first().drop(columns='rank_test_score').reset_index()


print(u"\\begin{table}[h]\n\\centering\n\\caption{Medida F1 para os parâmetros }")

print(re.sub(r'[ ]{2,}',' ',t.to_latex()))
print ("\label{tab:modelocaracter}")
print(r"\end{table}")

\begin{table}[h]
\centering
\caption{Medida F1 para os parâmetros }
\begin{tabular}{lllrlrl}
\toprule
{} & problem & language & mean\_test\_score & ngram\_range & rangeLen & parts \\
\midrule
0 & problem00001 & inglesa & 0.7964 & (2, 4) & 9 & ['PUNCT', 'PROPN', 'VERB', 'NOUN', 'ADV', 'ADJ', 'ADP', 'INTJ', 'PRON', 'DET', 'CCONJ', 'NUM', 'PART'] \\
1 & problem00002 & inglesa & 0.9771 & (3, 5) & 12 & ['PUNCT', 'VERB', 'ADV', 'PROPN', 'NOUN', 'NUM', 'ADJ', 'PRON', 'INTJ'] \\
2 & problem00003 & francesa & 0.7823 & (2, 4) & 9 & ['NOUN', 'VERB', 'PROPN', 'ADV', 'ADJ', 'PRON', 'PUNCT'] \\
3 & problem00004 & francesa & 1.0000 & (3, 5) & 12 & ['NOUN', 'ADV', 'VERB', 'PROPN', 'ADJ', 'X', 'SCONJ', 'DET', 'PRON', 'AUX', 'CCONJ', 'PUNCT', 'NUM'] \\
4 & problem00005 & italiana & 0.7042 & (2, 5) & 14 & ['VERB', 'NOUN', 'PUNCT', 'PROPN', 'ADV', 'ADJ', 'PRON'] \\
5 & problem00006 & italiana & 0.9194 & (3, 5) & 12 & ['VERB', 'PROPN', 'PUNCT', 'ADV', 'NOUN', 'PRON', 'SCONJ', 'AUX', 'ADJ', 'NUM', 'CONJ', '

In [126]:
# TESTS

In [139]:
def d(estimator, n_features=15):
    from IPython.display import Markdown, display, HTML
    names = np.array(estimator.named_steps['vect'].get_feature_names());
    classes_ = estimator.named_steps['clf'].classes_;
    weights = estimator.named_steps['clf'].coef_;
    
    def tag(tag, content, attrib=''):
        if attrib != '':
            attrib = ' style="' + attrib+'"'; 
        return ''.join(['<',tag,attrib,' >',content,'</',tag,'>']);
    
    def color(baseColor, intensity):
        r,g,b = baseColor[0:2],baseColor[2:4],baseColor[4:6]
        r,g,b = int(r, 16), int(g, 16), int(b, 16)
        
        f= (1-np.abs(intensity))/3;
        r = r + int((255-r)*f)
        g = g + int((255-g)*f)
        b = b + int((255-b)*f)
        rgb = '#%02x%x%x' % (r, g, b);
        #print(baseColor,rgb,r,g,b,intensity,f)
        return rgb
    
    
    spanStyle ='border-radius: 5px;margin:2px 4px;padding:2px; color:#FFF;display:inline-block;';
    
    lines = '<table>'+tag('thead',tag('th','Classes')+tag('th','positive')+tag('th','negative'))
    lines += '<tbody>'
    for i,c in enumerate(weights):
        c = np.round(c / np.abs(c).max(),2);
        positive = names[np.argsort(-c)][:n_features];
        positiveV = c[np.argsort(-c)][:n_features]
        negative = names[np.argsort(c)][:n_features];
        negativeV = c[np.argsort(c)][:n_features]
        
        lines += tag('tr',
                     tag('td', re.sub('\D0*','',classes_[i]))
                     + tag('td',''.join([tag('span',d.upper()+' '+str(v),spanStyle+'background:'+color('51A3DD',v)) for d,v in zip(positive,positiveV)]))
                     + tag('td',''.join([tag('span',d.upper()+' '+str(v),spanStyle+'background:'+color('DD5555',v)) for d,v in zip(negative,negativeV)]))
                    )
    lines+= '</tbody></table>'
        
    display(HTML(lines))
    #print(lines)
        
d(estimators[1])

1,"! 1.0 "" 0.52 ' 0.29 , 0.28 'V 0.28 'RE 0.27 'R 0.22 --- 0.21 - 0.18 'D 0.15 "" 0.13 'S 0.11 'LL 0.07 ( 0.07 'D 0.03","'M -0.25 'M -0.25 ) -0.21 'L -0.21 ! -0.14 * -0.14 - -0.14 --- -0.14 ' -0.1 * -0.09 ) -0.08 'RE -0.07 , -0.06 -- -0.04 'VE -0.04"
2,"'S 0.53 ) 0.36 ' 0.31 'LL 0.29 , 0.23 * 0.23 'LL 0.22 --- 0.19 * 0.18 'VE 0.17 'RE 0.07 'M 0.07 'R 0.07 ( 0.06 --- 0.0","! -1.0 'D -0.65 "" -0.58 "" -0.53 'V -0.37 'S -0.3 'RE -0.25 - -0.25 'M -0.19 ! -0.19 , -0.13 'VE -0.11 'L -0.08 ) -0.05 ( -0.04"
3,""" 1.0 'M 0.22 'D 0.17 ' 0.16 - 0.16 'VE 0.13 "" 0.08 'M 0.07 ( 0.06 -- 0.06 'RE 0.05 --- 0.05 , 0.03 'RE 0.03 , -0.0",! -0.41 ' -0.29 'VE -0.22 * -0.13 * -0.12 'R -0.12 'LL -0.11 - -0.09 ) -0.06 'S -0.06 'S -0.06 'L -0.06 --- -0.04 'D -0.03 ! -0.02
4,! 0.8 'D 0.49 ! 0.48 'L 0.48 ( 0.29 * 0.18 ) 0.16 * 0.12 'S 0.12 'VE 0.09 'V 0.09 'RE 0.04 ) 0.04 'M 0.03 - 0.02,""" -1.0 ( -0.54 'D -0.46 'RE -0.39 'S -0.38 , -0.35 ' -0.33 ' -0.24 , -0.22 'R -0.2 'LL -0.18 "" -0.17 'LL -0.17 --- -0.17 --- -0.14"
5,""" 0.86 ! 0.51 ' 0.42 ( 0.31 'S 0.27 'RE 0.26 ) 0.26 'M 0.25 'D 0.23 'M 0.18 - 0.16 , 0.09 --- 0.08 * 0.08 'VE 0.08",""" -1.0 ! -0.99 ) -0.3 'S -0.25 ' -0.22 ( -0.18 * -0.15 'LL -0.09 'RE -0.08 --- -0.07 'V -0.06 'LL -0.05 - -0.04 'L -0.02 -- 0.0"
